In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('data/uber_data.csv')

In [4]:
df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'])
df['tpep_dropoff_datetime'] = pd.to_datetime(df['tpep_dropoff_datetime'])

In [5]:
datetime_dim = df[['tpep_pickup_datetime' , 'tpep_dropoff_datetime']].drop_duplicates().reset_index(drop=True)
datetime_dim['pickup_hour'] = datetime_dim['tpep_pickup_datetime'].dt.hour
datetime_dim['pickup_day'] = datetime_dim['tpep_pickup_datetime'].dt.day
datetime_dim['pickup_month'] = datetime_dim['tpep_pickup_datetime'].dt.month
datetime_dim['pickup_year'] = datetime_dim['tpep_pickup_datetime'].dt.year
datetime_dim['pickup_weekday'] = datetime_dim['tpep_pickup_datetime'].dt.weekday

datetime_dim['dropoff_hour'] = datetime_dim['tpep_dropoff_datetime'].dt.hour
datetime_dim['dropoff_day'] = datetime_dim['tpep_dropoff_datetime'].dt.day
datetime_dim['dropoff_month'] = datetime_dim['tpep_dropoff_datetime'].dt.month
datetime_dim['dropoff_year'] = datetime_dim['tpep_dropoff_datetime'].dt.year
datetime_dim['dropoff_weekday'] = datetime_dim['tpep_dropoff_datetime'].dt.weekday

In [6]:
datetime_dim['datetime_id'] = datetime_dim.index

In [7]:
datetime_dim = datetime_dim [['datetime_id' , 'tpep_pickup_datetime' , 'pickup_hour' , 'pickup_day' , 'pickup_month' , 'pickup_year' , 'pickup_weekday',
              'tpep_dropoff_datetime' , 'dropoff_hour' , 'dropoff_day' , 'dropoff_month' , 'dropoff_year' , 'dropoff_weekday']]

In [8]:
passenger_count_dim = df[['passenger_count']].drop_duplicates().reset_index(drop=True)
passenger_count_dim['passenger_count_id'] = passenger_count_dim.index
passenger_count_dim = passenger_count_dim[['passenger_count_id' , 'passenger_count']]

trip_distance_dim = df[['trip_distance']].drop_duplicates().reset_index(drop=True)
trip_distance_dim['trip_distance_id'] = trip_distance_dim.index
trip_distance_dim = trip_distance_dim[['trip_distance_id' , 'trip_distance']]

In [9]:
rate_code_type = {
    
    1: "Standard",
    2: "JFK",
    3: "Newark",
    4: "Nassau or Westchester",
    5: "Negotiated Fare",
    6: "Group Ride"
}

rate_code_dim = df[["RatecodeID"]].drop_duplicates().reset_index(drop=True)
rate_code_dim['rate_code_id'] = rate_code_dim.index
rate_code_dim['rate_code_name'] = rate_code_dim['RatecodeID'].map(rate_code_type)
rate_code_dim = rate_code_dim[['rate_code_id' , 'RatecodeID' , 'rate_code_name']]

In [10]:
pickup_location_dim = df[['pickup_longitude' , 'pickup_latitude']].drop_duplicates().reset_index(drop=True)
pickup_location_dim['pickup_location_id'] = pickup_location_dim.index
pickup_location_dim = pickup_location_dim[['pickup_location_id' ,'pickup_longitude' , 'pickup_latitude']]

dropoff_location_dim = df[['dropoff_longitude' , 'dropoff_latitude']].drop_duplicates().reset_index(drop=True)
dropoff_location_dim['dropoff_location_id'] = dropoff_location_dim.index
dropoff_location_dim = dropoff_location_dim[['dropoff_location_id' ,'dropoff_longitude' , 'dropoff_latitude']]

In [11]:
payment_name = {
    1: "Credit Card",
    2: "Cash",
    3: "No charge",
    4: "Dispute", 
    5: "Unknown",
    6: "Voided trip"
}

payment_type_dim = df[['payment_type']].drop_duplicates().reset_index(drop=True)
payment_type_dim['payment_type_id'] = payment_type_dim.index
payment_type_dim['payment_type_name'] = payment_type_dim['payment_type'].map(payment_name)
payment_type_dim = payment_type_dim[['payment_type_id' , 'payment_type' , 'payment_type_name']]

In [12]:
fact_table = df.merge(passenger_count_dim, on='passenger_count') \
.merge(trip_distance_dim, on='trip_distance') \
.merge(rate_code_dim, on='RatecodeID') \
.merge(pickup_location_dim, on=['pickup_longitude' , 'pickup_latitude']) \
.merge(dropoff_location_dim, on=['dropoff_longitude' , 'dropoff_latitude']) \
.merge(datetime_dim, on=['tpep_pickup_datetime' , 'tpep_dropoff_datetime']) \
.merge(payment_type_dim, on='payment_type') \
            [['VendorID' , 'datetime_id' , 'passenger_count_id' , 'trip_distance_id' , 'rate_code_id',
              'store_and_fwd_flag' , 'pickup_location_id' , 'dropoff_location_id' , 'payment_type_id',
              'fare_amount' , 'extra' , 'mta_tax' , 'tip_amount' , 'tolls_amount' , 'improvement_surcharge' , 'total_amount']]

In [13]:
fact_table.head()

,VendorID,datetime_id,passenger_count_id,trip_distance_id,rate_code_id,store_and_fwd_flag,pickup_location_id,dropoff_location_id,payment_type_id,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,1,0,0,0,0,N,0,0,0,9.0,0.5,0.5,2.05,0.0,0.3,12.35
1,2,1491,0,0,0,N,1481,1484,0,10.5,0.0,0.5,2.26,0.0,0.3,13.56
2,2,2834,0,0,0,N,2816,2819,0,9.5,0.0,0.5,1.25,0.0,0.3,11.55
3,2,3488,0,0,0,N,3465,3470,0,13.5,0.0,0.5,2.00,0.0,0.3,16.30
4,2,3923,0,0,0,N,3899,3903,0,10.5,0.0,0.5,2.26,0.0,0.3,13.56


In [25]:
{"datetime_dim":datetime_dim.to_dict(orient="dict"),
    "passenger_count_dim":passenger_count_dim.to_dict(orient="dict"),
    "trip_distance_dim":trip_distance_dim.to_dict(orient="dict"),
    "rate_code_dim":rate_code_dim.to_dict(orient="dict"),
    "pickup_location_dim":pickup_location_dim.to_dict(orient="dict"),
    "dropoff_location_dim":dropoff_location_dim.to_dict(orient="dict"),
    "payment_type_dim":payment_type_dim.to_dict(orient="dict"),
    "fact_table":fact_table.to_dict(orient="dict")}

{'datetime_dim': {'datetime_id': {0: 0,
   1: 1,
   2: 2,
   3: 3,
   4: 4,
   5: 5,
   6: 6,
   7: 7,
   8: 8,
   9: 9,
   10: 10,
   11: 11,
   12: 12,
   13: 13,
   14: 14,
   15: 15,
   16: 16,
   17: 17,
   18: 18,
   19: 19,
   20: 20,
   21: 21,
   22: 22,
   23: 23,
   24: 24,
   25: 25,
   26: 26,
   27: 27,
   28: 28,
   29: 29,
   30: 30,
   31: 31,
   32: 32,
   33: 33,
   34: 34,
   35: 35,
   36: 36,
   37: 37,
   38: 38,
   39: 39,
   40: 40,
   41: 41,
   42: 42,
   43: 43,
   44: 44,
   45: 45,
   46: 46,
   47: 47,
   48: 48,
   49: 49,
   50: 50,
   51: 51,
   52: 52,
   53: 53,
   54: 54,
   55: 55,
   56: 56,
   57: 57,
   58: 58,
   59: 59,
   60: 60,
   61: 61,
   62: 62,
   63: 63,
   64: 64,
   65: 65,
   66: 66,
   67: 67,
   68: 68,
   69: 69,
   70: 70,
   71: 71,
   72: 72,
   73: 73,
   74: 74,
   75: 75,
   76: 76,
   77: 77,
   78: 78,
   79: 79,
   80: 80,
   81: 81,
   82: 82,
   83: 83,
   84: 84,
   85: 85,
   86: 86,
   87: 87,
   88: 88,
   89: 89,

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

